In [1]:
from vnstock3 import Vnstock
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime, timedelta
import copy

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [4]:
# Getting tickers from VN30

stock = Vnstock().stock(source='VCI')
vn30_tickers = stock.listing.symbols_by_group('VN30').values
vn30_tickers = np.delete(vn30_tickers, np.where(vn30_tickers == 'SSB')) # SSB only has data starting 2021 (not enough)
vn30_tickers

2025-01-19 17:49:07 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-01-19 17:49:07 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-01-19 17:49:07 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


array(['ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
       'HPG', 'MBB', 'MSN', 'MWG', 'PLX', 'POW', 'SAB', 'SHB', 'SSI',
       'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE'], dtype=object)

In [5]:
# Setting up start and end date for data
global_start = '2018-12-01'
global_end = '2025-01-01'

#  Gathering market data and bond data
market_data = Vnstock().stock(symbol = 'VNINDEX', source = 'VCI').\
quote.history(start = global_start, end = global_end, interval = '1M')
market_data['profit'] = (market_data['close'] - market_data['close'].shift(1)) / market_data['close'].shift(1) * 100
market_data.dropna(inplace = True)
bond_data = pd.read_csv('Vietnam_10_year_Government_Bond.csv')
bond_data.columns = ['time', 'close', 'open',' high', 'low', 'change']
bond_data['time'] = pd.to_datetime(bond_data['time'])
bond_data['profit'] = ((1 + bond_data['close']/100) ** (1/12) - 1) * 100

2025-01-19 17:49:08 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [6]:
quarter_start_months = {1: '01', 2: '04', 3: '07', 4: '10'}

class Company:
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        stock = Vnstock().stock(symbol = self.ticker, source = 'VCI')
        self.history = stock.quote.history(start = start, end = end, interval = '1M')
        
        self.history.loc[:, ['open', 'close', 'high', 'low']] = self.history[['open', 'close', 'high', 'low']] * 1000
        self.ratio = stock.finance.ratio(period='quarter', lang='en', dropna=True)
        self.balance = stock.finance.balance_sheet(period='quarter', lang='en', dropna=True)
        self.history['shares'] = np.nan

        # Tính market cap, và book value
        meta = self.ratio['Meta']
        meta['date'] = np.nan
        stats = self.ratio['Chỉ tiêu định giá']
        for i in meta.index:
            meta.loc[i, 'date'] = f'{meta['yearReport'][i]}-{quarter_start_months[meta['lengthReport'][i]]}-01'
        meta['date'] = pd.to_datetime(meta['date'])
        self.balance['date'] = np.nan
        for i in self.balance.index:
            self.balance.loc[i, 'date'] = f'{self.balance['yearReport'][i]}-{quarter_start_months[self.balance['lengthReport'][i]]}-01'
        self.balance.loc[:, 'date'] = pd.to_datetime(self.balance['date'])

        for date in meta['date'].index:
            if (meta.loc[date, 'date'] < pd.to_datetime(end)) and (meta.loc[date, 'date'] > pd.to_datetime(start)):
                temp_day = 0
                while True:
                    if len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] + timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    elif len(self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares']) == 1:
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'shares'] = stats.loc[date, 'Outstanding Share (Mil. Shares)']
                        self.history.loc[self.history['time'] == meta.loc[date, 'date'] - timedelta(days = temp_day), 'book_value'] = self.balance.loc[date, "OWNER'S EQUITY(Bn.VND)"]
                        break
                    temp_day += 1
        self.history = self.history.fillna(method = 'ffill').fillna(method = 'bfill')
        self.history['market_cap'] = self.history['shares'] * self.history['close']

        # Tính Book to Market Ratio
        self.history.loc[:, 'btm'] = self.history['book_value'] / self.history['market_cap']

        self.history.loc[:, 'hml'] = self.history['btm'].apply(lambda x : 'high' if x >= 1 else 'low')

        self.history.loc[:, 'monthly_profit'] = (self.history['close'] - self.history['close'].shift(1)) / self.history['close'].shift(1) * 100

        self.history = self.history.iloc[1:]

In [7]:
class Portfolio:
    def __init__(self, tickers, start, end, data = None):
        self.tickers = tickers  
        if data is None:
            self.data = {}
        else:
            self.data = data
        self.betas = []
        self.start = start
        self.end = end
        
    def create_profiles(self):
        for ticker in self.tickers:
            print(ticker)
            self.data[ticker] = Company(ticker, start = self.start, end = self.end)
        
    def create_sub_portfolio(self, sub_tickers):
        new_portfolio = Portfolio(tickers = sub_tickers, start = self.start, end = self.end)
        for ticker in sub_tickers:
            new_portfolio.data[ticker] = copy.deepcopy(self.data[ticker])
        return new_portfolio
    
    def evaluate_smb(self):
        aggregate_market_cap = pd.DataFrame([], columns= self.tickers)

        for ticker in self.tickers:
            self.data[ticker].history['smb'] = np.nan

        for index in self.data[self.tickers[0]].history.index:
            for ticker in self.tickers: 
                aggregate_market_cap.loc[index, ticker] = self.data[ticker].history.loc[index, 'market_cap']
                
        for index in aggregate_market_cap.index:
            percentiles = np.percentile(aggregate_market_cap.loc[index].values, [10, 30]) 
            for ticker in self.tickers:
                x = aggregate_market_cap.loc[index, ticker] 
                if x <= percentiles[0]:
                    self.data[ticker].history.loc[index, 'smb'] = 'small' 
                elif x <= percentiles[1]:
                    self.data[ticker].history.loc[index, 'smb'] = 'mid'
                else:
                    self.data[ticker].history.loc[index, 'smb'] = 'big'

    def setup_regression(self, market_data, bond_data):
        regression_data = pd.DataFrame([], columns= ['time', 'excess_return', 'mp', 'smb', 'hml'])
        regression_data['time'] = self.data[self.tickers[0]].history['time']
        regression_data = pd.merge(regression_data, bond_data[['time', 'profit']], on = 'time', how = 'left').fillna(method = 'ffill')
        regression_data['rf'] = regression_data['profit']
        regression_data.drop(columns = ['profit'], inplace = True)
        regression_data.index = self.data[self.tickers[0]].history.index
        
        for index in self.data[self.tickers[0]].history.index:
            date = regression_data.loc[index, 'time']
            rf = regression_data.loc[index, 'rf']
            smb = 0
            hml = 0

            temp_day = 0
                    
            mp = market_data.loc[index, 'profit'] - rf

            for ticker in self.tickers:
                if self.data[ticker].history.loc[index, 'smb'] == 'small':
                    smb += self.data[ticker].history.loc[index, 'monthly_profit']
                elif self.data[ticker].history.loc[index, 'smb'] == 'big':
                    smb -= self.data[ticker].history.loc[index, 'monthly_profit']
                if self.data[ticker].history.loc[index, 'hml'] == 'high':
                    hml += self.data[ticker].history.loc[index, 'monthly_profit']
                else:
                    hml -= self.data[ticker].history.loc[index, 'monthly_profit']
            
            regression_data.loc[index, 'smb'] = smb
            regression_data.loc[index, 'hml'] = hml
            regression_data.loc[index, 'mp'] = mp

        return regression_data
        
    def ff3_metrics(self, og_regression_data, use_data = 'newest', cutoff = None):
        # TÍNH EXCESS RETURN CHO TỪNG PORTFOLIO LẺ
        regression_data = copy.deepcopy(og_regression_data)

        for index in regression_data.index:
            total_return = np.mean([self.data[ticker].history.loc[index, 'monthly_profit'] for ticker in self.tickers]) - regression_data.loc[index, 'rf']
            regression_data.loc[index, 'excess_return'] = total_return

        if cutoff is None:
            cutoff = round((len(regression_data) * 0.8))
            
        train_set = regression_data[:cutoff]
        test_set = regression_data[cutoff:]

        y_train = train_set['excess_return']
        
        x_train = train_set.drop(columns = ['time', 'excess_return', 'rf'])

        y_test = test_set['excess_return']
        if use_data == 'newest':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])

        elif use_data == 'average':
            x_test = test_set.drop(columns = ['time', 'excess_return', 'rf'])
            for column in x_test.columns:
                m = np.mean(x_train[column])
                for index in x_test.index:
                    x_test.loc[index, column] = m

        model = LinearRegression()

        model.fit(x_train, y_train)
        
        y_train_pred = model.predict(x_train)
        
        mse_train = mean_squared_error(y_train, y_train_pred)

        y_pred = model.predict(x_test)
        mse_test = mean_squared_error(y_test, y_pred)
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_pred)

        return mse_train, mse_test, r2_train, r2_test, model.coef_

In [8]:
monthly_data = {}
for ticker in vn30_tickers:
    monthly_data[ticker] = Company(ticker, start = global_start, end = global_end)

2025-01-19 17:49:16 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:25 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:33 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:41 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:47 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:52 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:49:59 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:50:05 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất 

In [9]:
full_portfolio = Portfolio(vn30_tickers, start = global_start, end = global_end, data = monthly_data)
aggregate_market_cap = full_portfolio.evaluate_smb()
regression_data = full_portfolio.setup_regression(market_data, bond_data)

In [10]:
cutoff = 36

In [11]:
results = []

for ticker in vn30_tickers:
    temp = full_portfolio.create_sub_portfolio(sub_tickers= [ticker])
    
    _,_,_,_, [mp, smb, hml] = temp.ff3_metrics(regression_data, cutoff = cutoff)
    results.append([ticker, mp, smb, hml])

    del temp

In [12]:
# Sorting stocks by their coefficients in the Fama-French 3 factor model

mp_results = sorted(results, key = lambda x : x[1])
smb_results = sorted(results, key = lambda x : x[2])
hml_results = sorted(results, key = lambda x : x[3])

In [13]:
# Generating portfolios

import itertools

perms = [list(i) for i in list(itertools.product([0, 1], repeat=3))]

portfolios = []
for n in range(1,30):
    m = 30 - n
    for perm in perms:
        portfolio = []
        if perm[0] == 0:
            portfolio.extend([i[0] for i in mp_results][:n])
        else:
            portfolio.extend([i[0] for i in mp_results[::-1]][:m])
        if perm[1] == 0:
            portfolio.extend([i[0] for i in smb_results][:n])
        else:
            portfolio.extend([i[0] for i in smb_results[::-1]][:m])

        if perm[2] == 0:
            portfolio.extend([i[0] for i in hml_results][:n])
        else:
            portfolio.extend([i[0] for i in hml_results[::-1]][:m])

        portfolios.append(list(set(portfolio)))

In [14]:
monthly_test_data = pd.DataFrame([], columns = vn30_tickers)
for index in full_portfolio.data[vn30_tickers[0]].history.index:
    for ticker in vn30_tickers:
        monthly_test_data.loc[index, ticker] = full_portfolio.data[ticker].history.loc[index, 'monthly_profit']
monthly_test_data['date'] = full_portfolio.data['FPT'].history['time']
monthly_test_data = monthly_test_data.iloc[cutoff-1:].reset_index(drop = True)
monthly_test_data['profit'] = monthly_test_data.drop('date', axis = 1).mean(axis = 1)

In [15]:
# Explanation of coefficients:

# MP: Higher beta is good in bullish markets, lower beta is good in bearish market.
# SMB: Higher is more risky. Lower is safer
# HML: Higher is good in economic recoveries or in mature markets. Lower is good in strong bull market.

# 1 is high, 0 is low

In [16]:
def total_profit(history, col = 'profit'):
    total_profit = 1
    for index in history.index:
        total_profit = (total_profit + (total_profit * history.loc[index, col]) /100)
    return (total_profit - 1) * 100

def backtest_single(monthly_data, portfolio, log = False):
    budget_single = 10000000
    temp_df = pd.DataFrame([], columns = portfolio)

    for index in monthly_data.index:
        for ticker in portfolio:
            temp_df.loc[index, ticker] = monthly_data.loc[index, ticker]
    temp_df['portfolio_profit'] = temp_df.mean(axis = 1)
    temp_df['date'] = full_portfolio.data[vn30_tickers[0]].history['time'].iloc[cutoff-1:].reset_index(drop = True)

    for index in temp_df.index:
        budget_single = budget_single * (1 + temp_df.loc[index, 'portfolio_profit'] / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {temp_df.loc[index, 'portfolio_profit']}%    Current total assets: {budget_single}")
    profit_single  = ((budget_single - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_single}')
    return temp_df, profit_single

def backtest_diverse(monthly_data, portfolios, log = False):
    temp_dfs = []
    budget = 10000000

    history = pd.DataFrame([], columns = ['date', 'profit', 'total_value'])
    
    for portfolio in portfolios:
        temp_df, _ = backtest_single(monthly_data, portfolio)
        temp_dfs.append(temp_df)
    for index in temp_df.index:
        profit_month = np.mean([i.loc[index, 'portfolio_profit'] for i in temp_dfs if i is not np.nan])
        budget = budget * (1 + profit_month / 100)
        if log:
            print(f"Date: {temp_df.loc[index, 'date']}    Portfolio changed by: {profit_month}%    Current total assets: {budget}")
        history.loc[index, 'profit'] = profit_month
        history.loc[index, 'total_value'] = budget

    profit_full  = ((budget - 10000000) / 10000000) * 100
    if log:
        print(f'Trading finished. Total profit: {profit_full}')
    
    metrics = pd.DataFrame([], columns = ['percentage'])
    metrics.loc['ap'] = history.loc[history['profit'] > 0, 'profit'].mean()
    metrics.loc['al'] = history.loc[history['profit'] < 0, 'profit'].mean()
    metrics.loc['ar'] = history['profit'].mean()
    metrics.loc['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = temp_df['date'].reset_index(drop = True)
    metrics.loc['tr'] = total_profit(history)
    
    return history, metrics

In [17]:
vn30_data = Vnstock().stock(symbol = 'VN30', source = 'VCI').\
quote.history(start = global_start, end = global_end)
vnindex_data = Vnstock().stock(symbol = 'VNINDEX', source = 'VCI').\
quote.history(start = global_start, end = global_end)

start_2022 = vn30_data[((vn30_data['time'].dt.year == 2022) & (vn30_data['time'].dt.month == 1))].index.values[0]
end_2022 = vn30_data[((vn30_data['time'].dt.year == 2022) & (vn30_data['time'].dt.month == 12))].index.values[-1]

start_2023 = vn30_data[((vn30_data['time'].dt.year == 2023) & (vn30_data['time'].dt.month == 1))].index.values[0]
end_2023 = vn30_data[((vn30_data['time'].dt.year == 2023) & (vn30_data['time'].dt.month == 12))].index.values[-1]

start_2024 = vn30_data[((vn30_data['time'].dt.year == 2024) & (vn30_data['time'].dt.month == 1))].index.values[0]
end_2024 = vn30_data[((vn30_data['time'].dt.year == 2024) & (vn30_data['time'].dt.month == 12))].index.values[-1]

vn30_2022 = vn30_data[start_2022:end_2022]
vn30_2023 = vn30_data[start_2023:end_2023]
vn30_2024 = vn30_data[start_2024:end_2024]

vnindex_2022 = vnindex_data[start_2022:end_2022]
vnindex_2023 = vnindex_data[start_2023:end_2023]
vnindex_2024 = vnindex_data[start_2024:end_2024]

2025-01-19 17:51:45 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2025-01-19 17:51:45 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [26]:
history, metrics = backtest_diverse(monthly_test_data, portfolios)

print('Period 1: 2022 - 2023')
p1_2022 = total_profit(history[0:12])
p2_2022 = (vn30_2022['close'].iloc[-1] / vn30_2022['close'].iloc[0] - 1) * 100
p3_2022 = (vnindex_2022['close'].iloc[-1] / vnindex_2022['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2022}\n')
print(f'VN30 profit: {p2_2022}\n')
print(f'VNINDEX profit: {p3_2022}\n')

print('Period 2: 2023 - 2024:')
p1_2023 = total_profit(history[12:24])
p2_2023 = (vn30_2023['close'].iloc[-1] / vn30_2023['close'].iloc[0] - 1) * 100
p3_2023 = (vnindex_2023['close'].iloc[-1] / vnindex_2023['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2023}\n')
print(f'VN30 profit: {p2_2023}\n')
print(f'VNINDEX profit: {p3_2023}\n')

print('Period 3: 2024 - 2025:')
p1_2024 = total_profit(history[24:])
p2_2024 = (vn30_2024['close'].iloc[-1] / vn30_2024['close'].iloc[0] - 1) * 100
p3_2024 = (vnindex_2024['close'].iloc[-1] / vnindex_2024['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2024}\n')
print(f'VN30 profit: {p2_2024}\n')
print(f'VNINDEX profit: {p3_2024}\n')

print('Cumulative total: 2022 - 2025:')
print(f'Strategy profit: {total_profit(history)}\n')
print(f'VN30 profit: {(vn30_2024['close'].iloc[-1] / vn30_2022['close'].iloc[0] - 1) * 100}\n')
print(f'VNINDEX profit: {(vnindex_2024['close'].iloc[-1] / vnindex_2022['close'].iloc[0] - 1) * 100}\n')

print('Yearly average: 2022 - 2025:')
print(f'Strategy profit: {(p1_2022 + p1_2023 + p1_2024) / 3}\n')
print(f'VN30 profit: {(p2_2022 + p2_2023 + p2_2024) / 3}\n')
print(f'VNINDEX profit: {(p3_2022 + p3_2023 + p3_2024) / 3}\n')

Period 1: 2022 - 2023
Strategy profit: -22.28475843704052

VN30 profit: -35.318532013573936

VNINDEX profit: -33.8422108312904

Period 2: 2023 - 2024:
Strategy profit: 6.658553151599955

VN30 profit: 7.759369777989966

VNINDEX profit: 8.145416227608004

Period 3: 2024 - 2025:
Strategy profit: 22.39549411140045

VN30 profit: 19.06171573998796

VNINDEX profit: 12.397059343300466

Cumulative total: 2022 - 2025:
Strategy profit: 1.4535665997841951

VN30 profit: -13.568803043230027

VNINDEX profit: -16.620563982223146

Yearly average: 2022 - 2025:
Strategy profit: 2.256429608653295

VN30 profit: -2.8324821651986696

VNINDEX profit: -4.433245086793978



In [29]:
def backtest_adaptive(monthly_data, portfolios, log = False, pnum = 1):
    budget = 10000000
    temp_dfs = []

    for portfolio in portfolios:
        temp_df, _ = backtest_single(monthly_data, portfolio)
        temp_dfs.append([portfolio, temp_df])
    
    history = pd.DataFrame([], columns = ['date', 'profit'])
    chosen_portfolio = pd.Series([i[0] for i in sorted(temp_dfs, key = lambda x : x[1].iloc[0]['portfolio_profit'], reverse= True)[0:pnum]]).sum()
    
    for i in range(1, len(monthly_data)):
        profit = monthly_data.iloc[i][chosen_portfolio].mean()
        budget = budget * (1 + profit / 100)
        if log:
            print(f"{i}. Date: {monthly_data.iloc[i]['date']}    Portfolio: {chosen_portfolio}    Portfolio changed by: {profit}%    Current total assets: {budget}")
        chosen_portfolio = pd.Series([i[0] for i in sorted(temp_dfs, key = lambda x : x[1].iloc[i]['portfolio_profit'], reverse= True)[0:pnum]]).sum()
        history.loc[i, 'profit'] = profit
    if log:
        print(f'Trading finished. Total profit: {(budget - 10000000) / 10000000 * 100}')

    metrics = pd.DataFrame([], columns = ['percentage'])
    metrics.loc['ap'] = history.loc[history['profit'] > 0, 'profit'].mean()
    metrics.loc['al'] = history.loc[history['profit'] < 0, 'profit'].mean()
    metrics.loc['ar'] = history['profit'].mean()
    metrics.loc['wr'] = len(history[history['profit'] > 0]) / len(history)
    history = history.reset_index(drop= True)
    history['date'] = monthly_data[1:].reset_index()['date']
    metrics.loc['tr'] = total_profit(history)
    return history, metrics

In [30]:
history, metrics = backtest_adaptive(monthly_test_data, portfolios)

print('Period 1: 2022 - 2023')
p1_2022 = total_profit(history[0:12])
p2_2022 = (vn30_2022['close'].iloc[-1] / vn30_2022['close'].iloc[0] - 1) * 100
p3_2022 = (vnindex_2022['close'].iloc[-1] / vnindex_2022['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2022}\n')
print(f'VN30 profit: {p2_2022}\n')
print(f'VNINDEX profit: {p3_2022}\n')

print('Period 2: 2023 - 2024:')
p1_2023 = total_profit(history[12:24])
p2_2023 = (vn30_2023['close'].iloc[-1] / vn30_2023['close'].iloc[0] - 1) * 100
p3_2023 = (vnindex_2023['close'].iloc[-1] / vnindex_2023['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2023}\n')
print(f'VN30 profit: {p2_2023}\n')
print(f'VNINDEX profit: {p3_2023}\n')

print('Period 3: 2024 - 2025:')
p1_2024 = total_profit(history[24:])
p2_2024 = (vn30_2024['close'].iloc[-1] / vn30_2024['close'].iloc[0] - 1) * 100
p3_2024 = (vnindex_2024['close'].iloc[-1] / vnindex_2024['close'].iloc[0] - 1) * 100
print(f'Strategy profit: {p1_2024}\n')
print(f'VN30 profit: {p2_2024}\n')
print(f'VNINDEX profit: {p3_2024}\n')

print('Cumulative total: 2022 - 2025:')
print(f'Strategy profit: {total_profit(history)}\n')
print(f'VN30 profit: {(vn30_2024['close'].iloc[-1] / vn30_2022['close'].iloc[0] - 1) * 100}\n')
print(f'VNINDEX profit: {(vnindex_2024['close'].iloc[-1] / vnindex_2022['close'].iloc[0] - 1) * 100}\n')

print('Yearly average: 2022 - 2025:')
print(f'Strategy profit: {(p1_2022 + p1_2023 + p1_2024) / 3}\n')
print(f'VN30 profit: {(p2_2022 + p2_2023 + p2_2024) / 3}\n')
print(f'VNINDEX profit: {(p3_2022 + p3_2023 + p3_2024) / 3}\n')

Period 1: 2022 - 2023
Strategy profit: -38.44317745100294

VN30 profit: -35.318532013573936

VNINDEX profit: -33.8422108312904

Period 2: 2023 - 2024:
Strategy profit: 43.361584540216505

VN30 profit: 7.759369777989966

VNINDEX profit: 8.145416227608004

Period 3: 2024 - 2025:
Strategy profit: 22.743431027691695

VN30 profit: 19.06171573998796

VNINDEX profit: 12.397059343300466

Cumulative total: 2022 - 2025:
Strategy profit: 8.319649392477935

VN30 profit: -13.568803043230027

VNINDEX profit: -16.620563982223146

Yearly average: 2022 - 2025:
Strategy profit: 9.220612705635087

VN30 profit: -2.8324821651986696

VNINDEX profit: -4.433245086793978



In [51]:
history[12:24]

,date,profit
12,2023-01-31,15.897431
13,2023-02-28,-8.852599
14,2023-03-31,2.719498
15,2023-04-30,-0.295174
16,2023-05-31,2.832071
17,2023-06-30,7.005148
18,2023-07-31,20.081387
19,2023-08-31,6.412753
20,2023-09-30,-3.511331
21,2023-10-31,-14.214797


In [35]:
vn30_test_data = vn30_data[start_2022:]

In [72]:
moving_window_res = pd.DataFrame([], columns = ['year', 'month', 'profit', 'growth'])
for year in range(2022, 2025):
    if year == 2024 and month == 1:
        break
    for month in range(1,13):
        month_index = 12 * (year - 2022) + month - 1
        strategy_profit = total_profit(history[month_index:month_index+12])
        start_price = vn30_test_data[((vn30_test_data['time'].dt.year == year) & (vn30_data['time'].dt.month == month))].iloc[0]['close']
        end_price = vn30_test_data[((vn30_test_data['time'].dt.year == year+(month>1)) & (vn30_data['time'].dt.month == (13-month)))].iloc[-1]['close']
        growth = (end_price / start_price - 1) * 100
        moving_window_res.loc[len(moving_window_res)] = [year, month, strategy_profit, growth] 
        if year == 2024 and month == 1:
            break

In [73]:
moving_window_res

,year,month,profit,growth
0,2022.0,1.0,-38.443177,-35.518036
1,2022.0,2.0,-25.346894,-29.816707
2,2022.0,3.0,-29.412573,-31.625135
3,2022.0,4.0,-25.583151,-24.390102
4,2022.0,5.0,-18.566276,-11.158687
5,2022.0,6.0,-11.624379,-7.838322
6,2022.0,7.0,1.039187,-10.310324
7,2022.0,8.0,18.685358,-15.118010
8,2022.0,9.0,17.151964,-19.005508
9,2022.0,10.0,33.808015,-2.569873


In [21]:
# Specific metrics for each period: 
# For backtest_diverse, replace monthly_test_data with the period you want
# For backtest_adaptive, include one extra month at the start, as the first month is used to find the first portfolio